In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

!ls data

clean_bios.csv
clean_matches.csv
clean_stats.csv
ufc_moneylines.csv


# Money Lines

From [sportsbookreview](https://www.sportsbookreview.com/betting-odds/ufc/):
* When Conor McGregor fought Donald Cerrone at UFC 246 in Las Vegas, he was priced at -310 in the UFC Vegas odds. Cerrone was the +240 underdog. If you see a minus sign in UFC betting odds, it tells you how much you need to wager in order to earn a \\$100 profit. In this case, a \\$310 bet on McGregor would have netted you \\$100. A plus sign informs you of the profit you stand to make from a \\$100 bet. A \\$100 bet on Cowboy would have earned you a \\$240 profit if he won.
* **You don't need to understand moneyline conventions**. I already figured out how to get market-implied probabilities of each fighter winning. That's in `FighterOpener`, which will be our benchmark for the probability of `Fighter` winning the fight. 
* `FighterOpener`, `Date`, `FighterID`, and `OpponentID` are pretty much the only columns you want to worry about in this file. 
* Matches in `ufc_moneylines.csv` are only for UFC matches. 

`FighterID` is ESPN's unique identifier for MMA fighters. You can use it to join across `clean_matches.csv`, `clean_bios.csv`, and `clean_stats.csv`. 

In [2]:
ml_df = pd.read_csv("data/ufc_moneylines.csv")
print(ml_df.shape)
ml_df.head()

(2624, 25)


,missing,Date,FighterName,OpponentName,wager_A,wager_B,opener_A,opener_B,bet365_A,bet365_B,...,betway_B,FighterOpener,Opponent,FighterResult,Decision,Rnd,Time,Event,OpponentID,FighterID
0,False,2017-04-08,magomed bibulatov,jenel lausa,48%,52%,-145,+125,-500,+375,...,-,0.571116,Jenel Lausa,W,Decision - Unanimous,3.0,5:00,UFC 210: Cormier vs. Johnson 2,4058117/jenel-lausa,3151939/magomed-bibulatov
1,False,2017-04-08,irene aldana,katlyn chookagian,55%,45%,+145,-170,+130,-160,...,-,0.393299,Katlyn Chookagian,L,Decision - Split,3.0,5:00,UFC 210: Cormier vs. Johnson 2,4026490/katlyn-chookagian,3136286/irene-aldana
2,False,2017-04-08,desmond green,josh emmett,58%,42%,+190,-225,+180,-220,...,-,0.332481,Josh Emmett,W,Decision - Split,3.0,5:00,UFC 210: Cormier vs. Johnson 2,4011299/josh-emmett,3039667/desmond-green
3,False,2017-04-08,andrew holbrook,gregor gillespie,50%,50%,+185,-220,+200,-250,...,-,0.337909,Gregor Gillespie,L,KO/TKO,1.0,0:21,UFC 210: Cormier vs. Johnson 2,4021934/gregor-gillespie,3904497/andrew-holbrook
4,False,2017-04-08,jan blachowicz,patrick cummins,45%,55%,-125,+105,-125,+100,...,-,0.532468,Patrick Cummins,L,Decision - Unanimous,3.0,5:00,UFC 210: Cormier vs. Johnson 2,2536068/patrick-cummins,2506250/jan-blachowicz


# Matches

[ESPN keeps track](https://www.espn.com/mma/fighter/stats/_/id/3933168/francis-ngannou) of the outcomes of every fight/match. `clean_matches.csv` contains these results.

**All the matches in `ml_df` have already been joined with `match_df`, so you don't need this unless you want to look at fights for which I don't have odds info.**

# Match Stats

Every [fighter's page](https://www.espn.com/mma/fighter/stats/_/id/4350762/zhang-weili) tracks the following statistics for each match:

```
GLOSSARY
%BODY:Target Breakdown Body
%HEAD:Target Breakdown Head
%LEG:Target Breakdown Leg
AD:Advances
ADHG:Advance To Half Guard
ADTB:Advance To Back
ADTM:Advance To Mount
ADTS:Advance To Side
KD:Knockdowns
RV:Reversals
SCBA:Significant Clinch Body Strike Attempts
SCBL :Significant Clinch Body Strikes Landed
SCHA:Significant Clinch Head Strike Attempts
SCHL:Significant Clinch Head Strikes Landed
SCLA:Significant Clinch Leg Strike Attempts
SCLL:Significant Clinch Leg Strikes Landed
SDBL/A:Significant Distance Body Strikes Landed-Significant Distance Body Strike Attempts
SDHL/A:Significant Distance Head Strikes Landed-Significant Distance Head Strike Attempts
SDLL/A:Significant Distance Leg Strikes Landed-Significant Distance Leg Strike Attempts
SGBA :Significant Ground Body Strike Attempts
SGBL:Significant Ground Body Strikes Landed
SGHA:Significant Ground Head Strike Attempts
SGHL:Significant Ground Head Strikes Landed
SGLA:Significant Ground Leg Strikes Attempted
SGLL:Significant Ground Leg Strikes Landed
SM:Submissions
SR:Slam Rate
SSA:Significant Strikes Attempts
SSL:Significant Strikes Landed
TDA:Takedowns Attempted
TDL:Takedowns Landed
TDS:Takedowns Slams
TK ACC:Takedown Accuracy
TSA:Total Strikes Attempts
TSL:Total Strikes Landed
TSL-TSA:Total Strikes Landed-Total Strikes Attempts
```

In [3]:
stats_df = pd.read_csv("data/clean_stats.csv")
print(stats_df.shape)
stats_df.head()

(55668, 48)


,Date,Opponent,Event,FighterResult,SDBL/A,SDHL/A,SDLL/A,TSL,TSA,SSL,...,ADTS,SM,FighterID,SDBL,SDBA,SDHL,SDHA,SDLL,SDLA,Name
0,2010-10-09,Katja Kankaanpaa,BP 9,L,0/0,0/0,0/0,0.0,0.0,0.0,...,0.0,0.0,3043549/niina-aaltonen,0.0,0.0,0.0,0.0,0.0,0.0,niina aaltonen
1,2013-02-15,Brendan O'Reilly,Fury MMA 5,L,0/0,0/0,0/0,0.0,0.0,0.0,...,0.0,0.0,3089919/mike-aarts,0.0,0.0,0.0,0.0,0.0,0.0,mike aarts
2,2001-01-18,Josh Thomson,Bushido 1,L,0/0,0/0,0/0,0.0,0.0,0.0,...,0.0,0.0,2501542/jason-abajian,0.0,0.0,0.0,0.0,0.0,0.0,jason abajian
3,2015-10-30,Jermaine McDermott,TWC 23,L,0/0,0/0,0/0,0.0,0.0,0.0,...,0.0,0.0,3960654/marcus-abarquez,0.0,0.0,0.0,0.0,0.0,0.0,marcus abarquez
4,2006-09-23,Jorge Gurgel,UFC 63,L,0/1,41/121,0/0,79.0,173.0,56.0,...,0.0,0.0,2556806/danny-abbadi,0.0,1.0,41.0,121.0,0.0,0.0,danny abbadi


# Getting a benchmark score for predictions

How good of a predictor is the opening moneyline? Evaluate by cross-entropy loss. 

I think draws and no contests should be removed from consideration. If I were running a casino and a fighter was kicked out for doing something illegal, or getting popped for steroids, I would just give everybody their money back (casinos do this if you bet on a team and a star player gets in hurt before the game). 

In [4]:
ml_df["FighterResult"].value_counts()

L    1414
W    1010
D      51
Name: FighterResult, dtype: int64

In [5]:
from sklearn.metrics import log_loss

y_true = ml_df["FighterResult"] == "W"
y_pred = ml_df["FighterOpener"]
ignore_inds = y_true.isnull() | y_pred.isnull() | (ml_df["FighterResult"] == "D")
y_true = y_true.loc[~ignore_inds]
y_pred = y_pred.loc[~ignore_inds]

log_loss(y_true=y_true, y_pred=y_pred)

0.6220260010254359

# What I want from you

* Help me understand the relationship between these features, at the fighter level and the fight level. 
* **Something easy to start:**
    1. Calculate average of seemingly useful stats for each fighter (eg average SCBL per fight)
    2. Correlation heatmap of these fighter-level statistics
    3. Run PCA on the entire dataset of fighter-level statistics, and project onto the first two principal components. Make a scatterplot of fighters, embedded into this new space. Are there obvious clusters? 
* **If that was easy, here's what else you can do:**
    4. Run PCA on the entire dataset of fighter-level statistics, and look at the eigenvalues. Is there a clear "elbow point", suggesting an optimal number of principal components to use to describe fighters?
    5. Use first K principal components from 4, project the fighter-level statistics onto them, and run this projected data through your favorite clustering algorithm. What are the cluster centers like? Who are some "archetypical fighters" close to the centers of their respective clusters?
* Eventually I'll think of something you can do requiring you to join `ml_df` with `stats_df`.
* If you just get to step 2 I'll be happy tbh. 